[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.
## Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

## Objective
Objective of this dataset is base on **review** we predict **sentiment** (positive or negative) so X is **review** column and y is **sentiment** column

## 1. Load Dataset
we only forcus on "labeledTrainData.csv" file

Let's first of all have a look at the data.

[Click here to download dataset](https://s3-ap-southeast-1.amazonaws.com/ml101-khanhnguyen/week3/assignment/labeledTrainData.tsv)

In [1]:
# Import pandas, numpy
import pandas as pd
import numpy as np

In [2]:
# Read dataset with extra params sep='\t', encoding="latin-1"
BoW = pd.read_csv('labeledTrainData.tsv',sep='\t', encoding="latin-1")
BoW.sample(10)

,id,sentiment,review
14323,9778_2,0,I'm sure that Operations Dames was a favorite ...
16473,3375_9,1,In this movie the year 2022 looks much like th...
4043,7516_3,0,"Ya I rented it, so shoot me!<br /><br />A dece..."
20645,5304_10,1,Pecker is a hilariously funny yet twisted film...
5110,4718_7,1,This movie was made in Hungary i think. anyway...
24579,9610_4,0,"In Hollywood in the 1930's and 1940's, I think..."
19330,4599_2,0,The proverb \Never judge a book by it's cover\...
17039,2125_2,0,If I hadn't been forced to watch this for work...
15992,7303_10,1,I have only managed to see this classic for th...
6714,5135_10,1,Simply but imaginatively filmed studio-set per...


## 2. Preprocessing

In [9]:
# stop words
 #import nltk
 #nltk.download()
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [22]:
from collections import Counter
vocab = Counter()

# Let's apply the example above to count words in our SentimentText
# Your code here
for twit in BoW.review:
    for word in twit.split(' '):
        vocab[word] += 1

vocab.most_common(20)

[('the', 287025),
 ('a', 155092),
 ('and', 152651),
 ('of', 142970),
 ('to', 132568),
 ('is', 103227),
 ('in', 85576),
 ('I', 65971),
 ('that', 64555),
 ('this', 57195),
 ('it', 54416),
 ('/><br', 50935),
 ('was', 46697),
 ('as', 42509),
 ('with', 41718),
 ('for', 41067),
 ('but', 33783),
 ('The', 33760),
 ('on', 30765),
 ('movie', 30496)]

In [23]:
vocab_reduced = Counter()
# Go through all of the items of vocab using vocab.items() and pick only words that are not in 'stop' 
# and save them in vocab_reduced
# Your code here
for w, c in vocab.items():
    if not w in stop:
        vocab_reduced[w]=c

vocab_reduced.most_common(20)

[('I', 65971),
 ('/><br', 50935),
 ('The', 33760),
 ('movie', 30496),
 ('film', 27394),
 ('one', 20685),
 ('like', 18133),
 ('This', 12279),
 ('would', 11922),
 ('good', 11435),
 ('It', 10950),
 ('really', 10814),
 ('even', 10605),
 ('see', 10154),
 ('-', 9355),
 ('get', 8776),
 ('story', 8523),
 ('much', 8506),
 ('time', 7762),
 ('make', 7485)]

In [25]:
# Removing special characters and "trash"
import re
def preprocessor(text):
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text)
    
    # Save emoticons for later appending
    # Your code here
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    # Your code here
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    return text

In [29]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root
from nltk.stem import PorterStemmer
# Your code here
porter = PorterStemmer()
def tokenizer(text):
    return text.split()

def tokenizer_porter(text):
    token = []
    return [porter.stem(word) for word in text.split()] # Your code here
print(tokenizer_porter('wow,this is amazing'))

['wow,thi', 'is', 'amaz']


In [30]:
# split the dataset in train and test
# Your code here
from sklearn.model_selection import train_test_split

X = BoW['review']
y = BoW['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

## 3. Create Model and Train 

Using **Pipeline** to concat **tfidf** step and **LogisticRegression** step

In [31]:
# Import Pipeline, LogisticRegression, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words=stop,
                        tokenizer=tokenizer_porter,
                        preprocessor=preprocessor)



clf = Pipeline([('vect', tfidf),
                ('clf', LogisticRegression(random_state=0))])

clf.fit(X_train, y_train)

D:\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function prepr...e, penalty='l2', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])

## 4. Evaluate Model

In [32]:
# Using Test dataset to evaluate model
# classification_report
# confusion matrix
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Now apply those above metrics to evaluate your model
# Your code here
predictions = clf.predict(X_test)
print('accuracy:',accuracy_score(y_test,predictions))
print('confusion matrix:\n',confusion_matrix(y_test,predictions))
print('classification report:\n',classification_report(y_test,predictions))

accuracy: 0.8850666666666667
confusion matrix:
 [[3293  503]
 [ 359 3345]]
classification report:
               precision    recall  f1-score   support

           0       0.90      0.87      0.88      3796
           1       0.87      0.90      0.89      3704

   micro avg       0.89      0.89      0.89      7500
   macro avg       0.89      0.89      0.89      7500
weighted avg       0.89      0.89      0.89      7500



In [35]:
twits = [
    "Oh, it's pretty good",
    "something wrong!",
    ":(",
    "It looks disgusting",
    "it does not suit me",
    "it's so expensive",
]

preds = clf.predict_proba(twits)

for i in range(len(twits)):
    print(f'{twits[i]} --> Negative, Positive = {preds[i]}')

Oh, it's pretty good --> Negative, Positive = [0.87703694 0.12296306]
something wrong! --> Negative, Positive = [0.84553749 0.15446251]
:( --> Negative, Positive = [0.56271775 0.43728225]
It looks disgusting --> Negative, Positive = [0.89149148 0.10850852]
it does not suit me --> Negative, Positive = [0.62076641 0.37923359]
it's so expensive --> Negative, Positive = [0.62572189 0.37427811]


## 5. Export Model 

In [36]:
# Using pickle to export our trained model
import pickle
import os

pickle.dump(clf, open(os.path.join('data', 'logisticRegression.pkl'), 'wb'), protocol=4)

FileNotFoundError: [Errno 2] No such file or directory: 'data\\logisticRegression.pkl'